<a href="https://colab.research.google.com/github/kclassie/HSE_Open_Data_Cource/blob/main/%D0%98%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ТЕМА: возможности использования API (на примере проекта "Госрасходы" и ЕПБС)

Ссылка на проект "Госрасходы": https://spending.gov.ru/

Ссылка на документацию API: https://code.ach.gov.ru/sgr/spending-api-docs/-/wikis/home

##Инструменты##

Python 3 (https://www.python.org/downloads/)

Jupyter Notebook (https://jupyter.org/)

Google Colaboratoty (https://colab.research.google.com/notebooks/welcome.ipynb?hl=ru)

#1) Формирование запроса к API#

##1.1 Методы **GET**, **SEARCH**, **SELECT**##

- GET - получение сведений о конкретных контрактах по по регистрационному номеру (в выдаче - 1 контракт);
- SEARCH - полнотестовый поиск по контрактам, ограничение на выдачу - 500 контрактов;
- SELECT - поиск по точному значению (нет ограничения по выдаче).

**Параметры запроса можно (и нужно) сочетать**

**Пример 1**: поиск контрактов (всех, и 44ФЗ, и 223ФЗ) по коду региона заказчика
https://api.spending.gov.ru/v1/contracts/select/?customerregion=05

**Пример 2**: поиск контрактов  только по 44ФЗ и коду региона заказчика 
https://api.spending.gov.ru/v1/contracts/select/?fz=44&customerregion=05

**Пример 3**: поиск контрактов только по 44ФЗ и коду региона заказчика за 2020 год
https://api.spending.gov.ru/v1/contracts/select/?fz=44&customerregion=05&daterange=01.01.2020-01.01.2020




##1.2 Сортировка выдачи ##

**По цене контракта** - параметр price:

https://api.spending.gov.ru/v1/contracts/select/?fz=44&customerregion=05&sort=price (от меньшей к большей)

https://api.spending.gov.ru/v1/contracts/select/?fz=44&customerregion=05&sort=-price (от большей к меньшей)

**По дате подписания контракта** - параметр signDate

https://api.spending.gov.ru/v1/contracts/select/?fz=44&customerregion=05&sort=signDate (от ранних к поздним)

https://api.spending.gov.ru/v1/contracts/select/?fz=44&customerregion=05&sort=-signDate (от поздних к ранним)

## 2. Написание кода для запросов к API

**Requests** - python-библиотека для отправки всех видов HTTP-запросов.

In [ ]:
#импортируем библиотеки, которые нам пригодятся
#!pip install requests
#!pip install pandas

import requests            #чтобы сделать запросы к API
import pandas as pd         #чтобы обработать выдачу, сформировать и выгрузить таблицу

In [ ]:
#делаем запрос, результат (ответ API) сохраняем в переменную response

response = requests.get('https://api.spending.gov.ru/v1/contracts/select/?fz=44&customerregion=78&daterange=01.01.2022-12.02.2022')

#результат (ответ API) лучше проверить, особенно при больших выгрузках

print(response)


In [ ]:
#сохраняем везультат в переменную contracts с помощью метода json

contracts = response.json()

#смотрим, что получилось

len(contracts['contracts']['data'])

##3. Постраничная навигация##

**total** - найдено записей всего;

**page** - страница в выдаче;

**perpage** - количество записей в одном запросе (max - 50).\

Для перемещения по страницам добавляем к запросу параметр **page**.

https://api.spending.gov.ru/v1/contracts/select/?fz=44&customerregion=05&daterange=01.01.2020-01.01.2021&page=2 (3, 4, 5, 6...)

Сколько страниц нужно пройти, чтобы выгрузить все контракты? 
**total // 50 + 1**

In [ ]:
#получаем значение total из json-файла contracts

total = contracts['contracts']['total']
total

In [ ]:
#получаем количество страниц в выдаче 
pages = total // 50 + 1 #два бэкслеша - деление без остатка
pages

In [ ]:
# теперь можно создать цикл, который будет подставлять номер страницы в запрос и пройдет по всем страницам
# для этого используем функцию range и форматирование строки

for page in range(1, 360):
  response = requests.get(f'https://api.spending.gov.ru/v1/contracts/select/?fz=44&customerregion=05&daterange=01.01.2020-01.01.2021&page={page}')
  print(page)
  print(response)

##4. Парсинг JSON файла

**JSON (JavaScript Object Notation)** – текстовый формат для представления значений и объектов. Первоначально создан для JavaScript, но многие другие языки также имеют библиотеки, которые могут работать с ним. 

**Хранит структурированную информацию (ключ: значение). Доступен для чтения как программой, так и человеком.**

**По сути, JSON-файл - это вложенные в друг друга словари и списки. Словари в python обозначаются фигурными скобками {}, списки - квадратными скобками [ ]**.

Для написания корректного кода, необходимо пристально изучить структуру JSON-файла.

In [ ]:
#одна страница выдачи - это список из 50 контрактов

contracts['contracts']['data'][49]

#посмотрим на длину этого списка

#len(contracts['contracts']['data'])

#посмотрим на первый контракт в списке

#contracts['contracts']['data'][49]

In [ ]:
#получение необходимых данных 

# contracts['contracts']['data'][0]['regNum'] #регистрационный номер контракта
# contracts['contracts']['data'][0]['signDate'] #дата подписания контракта
# contracts['contracts']['data'][0]['customer'] #данные заказчика
contracts['contracts']['data'][0]['customer']['fullName'] #наименование заказчик
# contracts['contracts']['data'][0]['customer']['inn'] #ИНН заказчика
# contracts['contracts']['data'][0]['customer']['kpp'] #КПП заказчика
# contracts['contracts']['data'][0]['customer']['postalAddress'] #адрес заказчика

In [ ]:
#обработка множественных данных (вложенные списки)

#получение сведений о поставщиках

contracts['contracts']['data'][0]['suppliers'][0]['organizationName']
# contracts['contracts']['data'][0]['suppliers'][0] #данные об одном поставщике
# contracts['contracts']['data'][0]['suppliers'][0]['organizationName'] #наименование поставщика
# contracts['contracts']['data'][0]['suppliers'][0]['factualAddress'] #адрес поставщика
# contracts['contracts']['data'][0]['suppliers'][0]['inn']
# contracts['contracts']['data'][0]['suppliers'][0]['kpp']

In [ ]:
#получение сведений о продуктах
for prod in contracts['contracts']['data'][0]['products'][0]:
  print(contracts['contracts']['data'][0]['products'][0]['name']) #данные обо всех продуктах в контракте
# contracts['contracts']['data'][0]['products'][0] #данные об одном (первом) продукте в контракте
# contracts['contracts']['data'][0]['products'][0]['name'] #наименование
# contracts['contracts']['data'][0]['products'][0]['sum'] #сумма по продукту
# contracts['contracts']['data'][0]['products'][0]['quantity'] #количество
# contracts['contracts']['data'][0]['products'][0]['price'] #цена
# contracts['contracts']['data'][0]['products'][0]['OKPD2']['code'] #код ОКПД2

## Выгрузка данных ## 

Объединяем все предыдущие шаги в одном коде. Пример выгрузки "по продуктам".

In [ ]:
contracts_tab = []  #в этот список будем складывать информацию о каждом контракте

In [ ]:
for page in range(1, 350):
  response = requests.get(f'https://api.spending.gov.ru/v1/contracts/select/?fz=44&customerregion=05&daterange=01.01.2019-01.01.2020&page={page}')
  print(page)
  print(response)
  contracts = response.json()
  lst = contracts['contracts']['data']
  for cnt in lst:
    for prod in cnt['products']:
      one_cont = []
      one_cont.append(cnt['regNum'])
      one_cont.append(cnt['signDate'])
      one_cont.append(cnt['price'])
      one_cont.append(cnt['customer']['fullName'])
      one_cont.append(cnt['customer']['inn'])
      one_cont.append(cnt['customer']['kpp'])
      suppliers = []
      for sup in cnt['suppliers']:
        suppliers.append(sup['organizationName'])
        try:
          suppliers.append(sup['factualAddress'])
        except KeyError:
          suppliers.append('None')
        suppliers.append(sup['inn'])
        try:
          suppliers.append(sup['kpp'])
        except KeyError:
          suppliers.append('None')
      one_cont.extend(suppliers)
      one_cont.append(prod['name'])
      try:
        one_cont.append(prod['sum'])
      except KeyError:
        one_cont.append('None')
      try:
        one_cont.append(prod['quantity'])
      except KeyError:
        one_cont.append('None')
      one_cont.append(prod['price'])
      try:
        one_cont.append(prod['OKPD2']['code'])
      except KeyError:
        one_cont.append('None')
      contracts_tab.append(one_cont)

#этот код выдаст ошибку, так как не найдет ключ - необходима обработка исключений (см. следующую ячейку)

In [ ]:
# проверим выгруженные значения в итоговом списке

contracts_tab


In [ ]:
# с помощью метода pd.DataFrame преобразуем список в датафрейм (таблицу)

frame = pd.DataFrame(contracts_tab)

In [ ]:
frame

In [ ]:
frame = frame.drop(columns=[15, 16, 17])

In [ ]:
#посмотрим, что получилось

frame.head(50)

In [ ]:
#зададим названия столбцам

frame.rename(columns={0: 'regnum', 1: 'sign_date', 2: 'contract_price', 3: 'customer_name', 4: 'customer_inn', 5: 'customer_kpp',
                      6: 'supplier_name', 7: 'supplier_adress', 8: 'supplier_inn', 9: 'supplier_kpp', 10: 'product_name', 11: 'product_sum',
                      12: 'product_quantity', 13: 'product_price', 14: 'product_OKPD2'}, inplace=True)

In [ ]:
#сохраним выгрузку в таблицу формата csv

frame.to_csv('contracts.csv', index=False)

##На примере Единого портала бюджетной системы: ##

http://budget.gov.ru/epbs/faces/p/%D0%94%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%B8%20%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/opendata?_adf.ctrl-state=14nlb34tpg_38&regionId=61

Набор: "Информация об основных параметрах реализации региональных проектов"

http://budget.gov.ru/epbs/faces/p/%D0%94%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%B8%20%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/opendata?code=7710168360-REGIONALPROJECT&_adf.ctrl-state=14nlb34tpg_95&regionId=61

In [ ]:
#формируем запрос

response = requests.get('http://budget.gov.ru/epbs/registry/7710168360-REGIONALPROJECT/data')
projects_data = response.json()

In [ ]:
#изучаем структуру JSON

projects_data['data'][0]

In [ ]:
#запускаем цикл для выгрузки данных
region_projects = []

for proj in projects_data['data']:
  proj_data = []
  proj_data.append([proj['fpcode'], proj['fpname'], proj['fullname'], proj['startdateproj'], proj['enddateproj'],
                   proj['curator'], proj['subject']['code'], proj['subject']['name']])
  region_projects.extend(proj_data)


In [ ]:
#посмотрим, что получилось

region_projects

In [ ]:
# с помощью метода pd.DataFrame преобразуем список в датафрейм (таблицу)

frame = pd.DataFrame(region_projects)
frame